In [9]:
import requests, time, random, numpy, re
from bs4 import BeautifulSoup, Comment
from teams import _tms
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from geopy.distance import geodesic
import pytz
from zoneinfo import ZoneInfo

In [136]:
def get_boxscore_urls(year):
    """
    Scrape boxscore URLs for all regular season games in a given NFL year from Pro-Football-Reference.
    """
    time.sleep(random.uniform(1.0, 2.0))
    base_url = f"https://www.pro-football-reference.com/years/{year}/games.htm"
    res = requests.get(base_url)
    soup = BeautifulSoup(res.content, 'html.parser')
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    

    game_table = soup.find('table', id='games')

    if game_table is None:
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        for comment in comments:
            if 'id="games"' in comment:
                comment_soup = BeautifulSoup(comment, 'html.parser')
                game_table = comment_soup.find('table', id='games')
                break

    if not game_table:
        print(f"⚠️ Could not find game table for {year}. Skipping.")
        return []
    links = []

    for row in game_table.tbody.find_all('tr'):
        if row.get('class') == ['thead']:
            continue  # skip header rows

        boxscore_cell = row.find('td', {'data-stat': 'boxscore_word'})

        # Safe check for <a> tag and href
        if boxscore_cell:
            a_tag = boxscore_cell.find('a')
            if a_tag and 'boxscores' in a_tag['href']:
                full_url = 'https://www.pro-football-reference.com' + a_tag['href']
                links.append(full_url)

    return links

def _season_and_event_date(
    boxscore_stats_link,
):

    date_regex =  re.findall(
        r'\d+',
        boxscore_stats_link
    )

    year = date_regex[0][0:4]
    month = date_regex[0][4:6]
    day = date_regex[0][6:8]

    if month == '01' or month == 1:
        season = int(year) - 1
        event_date = f'{year}-{month}-{day}'
    else:
        season = int(year)
        event_date = f'{year}-{month}-{day}'

    return {
        'season': season,
        'event_date': event_date,
    }


def _standardized(
    reference_team,
):

    tms = _tms()

    tm_tags = (
        'nano',
        'market',
        'name',
        'alias',
    )

    standardized = {
        tm_tag: tms.get(
            reference_team,
        ).get(
            tm_tag
        )
        for tm_tag in tm_tags
    }

    return standardized


def _reference_team(
    stat_text,
    thead, 
):

    reference_team = thead.find(
        attrs={'data-stat': stat_text}
    ).text
    reference_team = reference_team.lower()
    
    return reference_team

def extract_final_scores_from_scoring_table(soup):
    scoring_table = soup.find("table", {"id": "scoring"})
    if not scoring_table:
        return None, None, None, None

    headers = scoring_table.find_all("th")
    team_headers = [th.text.strip().lower() for th in headers if th.text.strip() not in ["Quarter", "Time", "Tm", "Detail"]]
    if len(team_headers) < 2:
        return None, None, None, None

    team1, team2 = team_headers[0], team_headers[1]

    final_row = scoring_table.find_all("tr")[-1]
    cells = final_row.find_all("td")
    if len(cells) < 2:
        return None, None, None, None

    try:
        team1_score = int(cells[-2].text.strip())
        team2_score = int(cells[-1].text.strip())
    except:
        return None, None, None, None

    return team1, team1_score, team2, team2_score

def _epa_stats(soup, comments):
    expected_points_by_team = {}

    expected_points_table = soup.find("table", {"id": "expected_points"})

    if not expected_points_table:
        for comment in comments:
            if 'id="expected_points"' in comment:
                comment_soup = BeautifulSoup(comment, 'html.parser')
                expected_points_table = comment_soup.find("table", {"id": "expected_points"})
                if expected_points_table:
                    break

    teams_seen = []

    if expected_points_table:
        for row in expected_points_table.find_all("tr"):
            cells = row.find_all("td")
            if len(cells) < 8:
                continue

            team_name = row.find_previous("th").text.strip().lower()
            if team_name not in teams_seen:
                teams_seen.append(team_name)

            # 🔁 Assign home_stat to the first team, vis_stat to the second
            stat_side = "home_stat" if len(teams_seen) == 1 else "vis_stat"

            try:
                expected_points_by_team[stat_side] = {
                    "epa_total": float(cells[0].text.strip()),
                    "epa_offense_total": float(cells[1].text.strip()),
                    "epa_offense_pass": float(cells[2].text.strip()),
                    "epa_offense_rush": float(cells[3].text.strip()),
                    "epa_defense_total": float(cells[5].text.strip()),
                    "epa_defense_pass": float(cells[6].text.strip()),
                    "epa_defense_rush": float(cells[7].text.strip()),
                }
            except:
                expected_points_by_team[stat_side] = {
                    "epa_total": None,
                    "epa_offense_total": None,
                    "epa_offense_pass": None,
                    "epa_offense_rush": None,
                    "epa_defense_total": None,
                    "epa_defense_pass": None,
                    "epa_defense_rush": None,
                }

    return expected_points_by_team


def _rush_stats(
    stat_text,
    tbody, 
):

    try:
        table_body_row = tbody[1]
        rush_stats_text = table_body_row.find(
            attrs={'data-stat': stat_text}
        ).text
        rush_stats_text_split = rush_stats_text.replace('--', '-')
        rush_stats_text_split = rush_stats_text_split.split('-')
        rush_att = rush_stats_text_split[0]
        rush_att = int(rush_att)
        rush_yds = rush_stats_text_split[1]
        rush_yds = int(rush_yds)
        rush_tds = rush_stats_text_split[2]
        rush_tds = int(rush_tds)
        return {
            'rush_att': rush_att,
            'rush_yds': rush_yds,
            'rush_tds': rush_tds,
        }
    except ValueError:
        return {
            'rush_att': None,
            'rush_yds': None,
            'rush_tds': None,
        }
    

def _pass_stats(
    stat_text,
    tbody, 
):

    try:
        table_body_row = tbody[2]
        pass_stats_text = table_body_row.find(
            attrs={'data-stat': stat_text}
        ).text
        pass_stats_text_split = pass_stats_text.replace('--', '-')
        pass_stats_text_split = pass_stats_text_split.split('-')
        pass_cmp = pass_stats_text_split[0]
        pass_cmp = int(pass_cmp)
        pass_att = pass_stats_text_split[1]
        pass_att = int(pass_att)
        pass_yds = pass_stats_text_split[2]
        pass_yds = int(pass_yds)
        pass_tds = pass_stats_text_split[3]
        pass_tds = int(pass_tds)
        pass_int = pass_stats_text_split[4]
        pass_int = int(pass_int)

        try:
            pass_cmp_pct = round((pass_cmp / pass_att), 3)
        except ZeroDivisionError:
            pass_cmp_pct = 0

        return {
            'pass_cmp': pass_cmp,
            'pass_att': pass_att,
            'pass_cmp_pct': pass_cmp_pct,
            'pass_yds': pass_yds,
            'pass_tds': pass_tds,
            'pass_int': pass_int,
        }

    except ValueError:
        return {
            'pass_cmp': None,
            'pass_att': None,
            'pass_cmp_pct': None,
            'pass_yds': None,
            'pass_tds': None,
            'pass_int': None,
        }


def _passer_rating(
    pass_stats,
):

    pass_cmp = pass_stats.get(
        'pass_cmp'
    )
    pass_att = pass_stats.get(
        'pass_att'
    )
    pass_yds = pass_stats.get(
        'pass_yds'
    )
    pass_tds = pass_stats.get(
        'pass_tds'
    )
    pass_int = pass_stats.get(
        'pass_int'
    )

    blocks = {
        'a': (pass_cmp / pass_att - .3) * 5,
        'b': (pass_yds / pass_att - 3) * .25,
        'c': pass_tds / pass_att * 20,
        'd': 2.375 - pass_int / pass_att * 25,
    }
    blocks = {
        block: min((value, 2.375))
        for block, value in blocks.items()
    }
    blocks = {
        block: max((value, 0))
        for block, value in blocks.items()
    }
    passer_rating = sum(blocks.values()) / 6 * 100
    passer_rating = round(
        passer_rating,
        3,
    )

    return {
        'passer_rating': passer_rating
    }


def _sack_stats(
    stat_text,
    tbody, 
):

    try:
        table_body_row = tbody[3]
        sack_stats_text = table_body_row.find(
            attrs={'data-stat': stat_text}
        ).text
        sack_stats_text_split = sack_stats_text.split('-')
        times_sacked = sack_stats_text_split[0]
        times_sacked = int(times_sacked)
        yds_sacked_for = sack_stats_text_split[1]
        yds_sacked_for = int(yds_sacked_for)
        return {
            'times_sacked': times_sacked,
            'yds_sacked_for': yds_sacked_for,
        }
    except ValueError:
        return {
            'times_sacked': None,
            'yds_sacked_for': None,
        }


def _total_yds(
    stat_text,
    tbody, 
):

    try:
        table_body_row = tbody[5]
        total_yds = table_body_row.find(
            attrs={'data-stat': stat_text}
        ).text
        total_yds = int(total_yds)
        return {
            'total_yds': total_yds,
        }          
    except ValueError:
        return {
            'total_yds': None,
        }   


def _fumbles_stats(
    stat_text,
    tbody, 
):

    try:
        table_body_row = tbody[6]
        sack_stats_text = table_body_row.find(
            attrs={'data-stat': stat_text}
        ).text
        sack_stats_text_split = sack_stats_text.split('-')
        fumbles = sack_stats_text_split[0]
        fumbles = int(fumbles)
        fumbles_lost = sack_stats_text_split[1]
        fumbles_lost = int(fumbles_lost)
        return {
            'fumbles': fumbles,
            'fumbles_lost': fumbles_lost,
        }   
    except ValueError:
        return {
            'fumbles': None,
            'fumbles_lost': None,
        }   


def _turnover_stats(
    stat_text,
    tbody, 
):

    try:
        table_body_row = tbody[7]
        turnovers = table_body_row.find(
            attrs={'data-stat': stat_text}
        ).text
        turnovers = int(turnovers)
        return {
            'turnovers': turnovers,
        }          
    except ValueError:
        return {
            'turnovers': None,
        }  


def _penalty_stats(
    stat_text,
    tbody, 
):

    try:
        table_body_row = tbody[8]
        penalty_stats_text = table_body_row.find(
            attrs={'data-stat': stat_text}
        ).text
        penalty_stats_text_split = penalty_stats_text.split('-')
        penalties = penalty_stats_text_split[0]
        penalties = int(penalties)
        penalty_yds = penalty_stats_text_split[1]
        penalty_yds = int(penalty_yds)
        return {
            'penalties': penalties,
            'penalty_yds': penalty_yds,
        }  
    except ValueError:
        return {
            'penalties': None,
            'penalty_yds': None,
        }  

def _time_of_possession(
    stat_text,
    tbody, 
):

    try:
        table_body_row = tbody[11]
        time_of_possession_text = table_body_row.find(
            attrs={'data-stat': stat_text}
        ).text
        minutes, seconds = time_of_possession_text.split(':')
        minutes = int(minutes)
        seconds = int(seconds)
        time_of_possession = (minutes * 60) + seconds
        return {
            'time_of_possession': time_of_possession,
        }
    except ValueError:
        return {
            'time_of_possession': None,
        }


def _gamelog_statistics(
    boxscore_stats_link,
):

    print('~'*100)
    print(f'Processing Gamelog Statistics For: {boxscore_stats_link}')
    print('~'*100)

    try:
        season_and_event_date = _season_and_event_date(
            boxscore_stats_link,
        )
    except IndexError:
        print('~'*100)
        print('Unable to Process Gamelog Statistics - Check Boxscore Stats Link Formatting')
        print('~'*100)
        return pd.DataFrame()
    
    try:
        time.sleep(random.uniform(1.0, 1.8))
        res = requests.get(
            boxscore_stats_link
        )
    except requests.exceptions.RequestException:
        print('~'*100)
        print('Request Exception - Check Boxscore Stats Link Formatting')
        print('~'*100)
        return pd.DataFrame()

    dlist = []

    if res.status_code == 200:        

        sleeptime = random.uniform(
            1.0, 
            1.8,
        )
        time.sleep(
            sleeptime
        )

        soup = BeautifulSoup(
            res.content, 
            'html.parser'
        )
        comments = soup.find_all(
            string = lambda text: isinstance(text, Comment)
        )
        team1, team1_score, team2, team2_score = extract_final_scores_from_scoring_table(soup)
        team_pts_map = {team1: team1_score, team2: team2_score}
        expected_points_by_team = _epa_stats(soup,comments)


        for comment in comments:
            
            if 'id="team_stats"' in comment.extract():

                comment_soup = BeautifulSoup(
                    comment.extract(), 
                    'html.parser'
                )
                thead = comment_soup.find(
                    'thead'
                ).find(
                    'tr'
                )
                tbody = comment_soup.find(
                    'tbody'
                ).find_all(
                    'tr'
                )
                
                for stat_side in ['vis_stat', 'home_stat']:
                    
                    reference_team = _reference_team(
                        stat_side,
                        thead,
                    )
                    reference_team = reference_team.lower()
                    standardized = _standardized(
                        reference_team,
                    )
                    team_pts = team_pts_map.get(reference_team, None
                    )
                    epa = expected_points_by_team.get(stat_side, {
                        "epa_total": None,
                        "epa_offense_total": None,
                        "epa_offense_pass": None,
                        "epa_offense_rush": None,
                        "epa_defense_total": None,
                        "epa_defense_pass": None,
                        "epa_defense_rush": None
                    })
                    rush_stats = _rush_stats(
                        stat_side,
                        tbody,
                    )
                    pass_stats = _pass_stats(
                        stat_side,
                        tbody,
                    )
                    passer_rating = _passer_rating(
                        pass_stats,
                    )                    
                    total_yds = _total_yds(
                        stat_side,
                        tbody,
                    )
                    sack_stats = _sack_stats(
                        stat_side,
                        tbody,
                    )
                    turnover_stats = _turnover_stats(
                        stat_side,
                        tbody,
                    )
                    penalty_stats = _penalty_stats(
                        stat_side,
                        tbody,
                    )                    
                    time_of_possession = _time_of_possession(
                        stat_side,
                        tbody,
                    )                    
                    
                    statistics = {
                        **season_and_event_date,
                        **standardized,
                        "points": team_pts,
                        **epa,
                        **rush_stats,
                        **pass_stats,
                        **passer_rating,
                        **total_yds,
                        **sack_stats,
                        **turnover_stats,
                        **penalty_stats,
                        **time_of_possession,
                        "boxscore_stats_link": boxscore_stats_link,
                    }

                    dlist.append(
                        statistics
                    )

    dset = pd.DataFrame(
        dlist
    )
    
    dset = dset.replace({
        numpy.nan: None
    })

    return dset

In [140]:
if __name__ == "__main__":
    all_stats = []

    for year in range (2015,2025):
        time.sleep(random.uniform(5, 7))
        urls = get_boxscore_urls(year)
        print(f"\nScraping {len(urls)} games for {year}...")

        for url in tqdm(urls, desc=f"Scraping {year}"):
            try:
                game_df = _gamelog_statistics(url)

                if not game_df.empty:
                    game_df = game_df.reset_index(drop=True)
                    game_df['home_away'] = ['away', 'home']
                    all_stats.append(game_df)
            except Exception as e:
                print(f"Error processing {url}: {e}")

    full_df = pd.concat(all_stats, ignore_index=True)

    full_df['event_date'] = pd.to_datetime(full_df['event_date'])
    full_df = full_df.sort_values(by=['market', 'event_date'])
    full_df['days_since_last_game'] = full_df.groupby('alias')['event_date'].diff().dt.days
    full_df['short_rest'] = full_df['days_since_last_game'] < 5

    full_df.to_csv("nfl_stats.csv", index=False)
    print("✅ All stats saved to nfl_stats.csv")


Scraping 267 games for 2015...


Scraping 2015:   0%|                                    | 0/267 [00:00<?, ?it/s]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Gamelog Statistics For: https://www.pro-football-reference.com/boxscores/201509100nwe.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Scraping 2015:   0%|                            | 1/267 [00:03<15:16,  3.45s/it]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Gamelog Statistics For: https://www.pro-football-reference.com/boxscores/201509130chi.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


...

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Gamelog Statistics For: https://www.pro-football-reference.com/boxscores/202501260kan.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Scraping 2024: 100%|█████████████████████████▉| 284/285 [15:07<00:03,  3.28s/it]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Gamelog Statistics For: https://www.pro-football-reference.com/boxscores/202502090phi.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Scraping 2024: 100%|██████████████████████████| 285/285 [15:11<00:00,  3.20s/it]


✅ All stats saved to nfl_stats.csv


In [166]:
full_df

,season,event_date,nano,market,name,alias,points,epa_total,epa_offense_total,epa_offense_pass,...,times_sacked,yds_sacked_for,turnovers,penalties,penalty_yds,time_of_possession,boxscore_stats_link,home_away,days_since_last_game,short_rest
17,2015,2015-09-13,GJizlkzQFaqGA,Arizona,Cardinals,ARI,31,5.03,12.48,24.95,...,0,0,1,5,30,1596,https://www.pro-football-reference.com/boxscor...,home,14.0,False
34,2015,2015-09-20,GJizlkzQFaqGA,Arizona,Cardinals,ARI,48,25.00,17.96,16.94,...,0,0,2,8,58,1804,https://www.pro-football-reference.com/boxscor...,away,7.0,False
87,2015,2015-09-27,GJizlkzQFaqGA,Arizona,Cardinals,ARI,47,40.00,21.61,17.48,...,1,4,1,4,24,2192,https://www.pro-football-reference.com/boxscor...,home,7.0,False
117,2015,2015-10-04,GJizlkzQFaqGA,Arizona,Cardinals,ARI,22,-2.00,5.76,6.02,...,4,18,3,3,20,1993,https://www.pro-football-reference.com/boxscor...,home,7.0,False
144,2015,2015-10-11,GJizlkzQFaqGA,Arizona,Cardinals,ARI,42,-25.00,-9.65,-1.43,...,2,13,0,9,70,1408,https://www.pro-football-reference.com/boxscor...,away,7.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5425,2024,2024-12-29,9JtBk8kFeaGEs,Washington,Football Team,WAS,30,-6.00,10.20,6.32,...,5,31,1,13,108,2492,https://www.pro-football-reference.com/boxscor...,home,7.0,False
5440,2024,2025-01-05,9JtBk8kFeaGEs,Washington,Football Team,WAS,23,3.00,5.93,2.59,...,6,43,1,5,55,1377,https://www.pro-football-reference.com/boxscor...,away,7.0,False
5468,2024,2025-01-12,9JtBk8kFeaGEs,Washington,Football Team,WAS,23,3.00,11.83,18.52,...,1,0,0,5,39,2126,https://www.pro-football-reference.com/boxscor...,away,7.0,False
5474,2024,2025-01-18,9JtBk8kFeaGEs,Washington,Football Team,WAS,45,14.00,27.77,23.49,...,0,0,0,4,25,2039,https://www.pro-football-reference.com/boxscor...,away,6.0,False


In [168]:
home_df = full_df[full_df['home_away'] == 'home'].copy()
away_df = full_df[full_df['home_away'] == 'away'].copy()
home_df = home_df.drop(columns=['season', 'event_date'], errors='ignore')

home_df = home_df.add_prefix('home_')
away_df = away_df.add_prefix('away_')

merged_df = pd.merge(
    home_df, 
    away_df, 
    left_on='home_boxscore_stats_link', 
    right_on='away_boxscore_stats_link', 
    suffixes=('', '_away')
)


merged_df.drop(columns=['away_boxscore_stats_link'], inplace=True)

merged_df.rename(columns={'home_boxscore_stats_link': 'boxscore_stats_link'}, inplace=True)
merged_df.rename(columns={'away_season': 'season'}, inplace=True)
merged_df.rename(columns={'away_event_date': 'event_date'}, inplace=True)
merged_df = merged_df.sort_values(by=['away_market', 'event_date'])
merged_df['days_since_last_game'] = merged_df.groupby('alias')['event_date'].diff().dt.days

merged_df['days_since_last_game'] = merged_df['days_since_last_game'].apply(
    lambda x: 14 if pd.isna(x) or x > 21 else x
)



merged_df.to_csv("nfl_stats.csv", index=False)
home_df.to_csv("home_stats.csv", index=False)
away_df.to_csv("away_stats.csv", index=False)
merged_df

,home_nano,home_market,home_name,home_alias,home_points,home_epa_total,home_epa_offense_total,home_epa_offense_pass,home_epa_offense_rush,home_epa_defense_total,...,away_times_sacked,away_yds_sacked_for,away_turnovers,away_penalties,away_penalty_yds,away_time_of_possession,away_home_away,away_days_since_last_game,away_short_rest,days_since_last_game
431,1vVmWGzUZ1rea,Chicago,Bears,CHI,23,-25.00,-9.19,-6.02,-3.09,-17.96,...,0,0,2,8,58,1804,away,7.0,False,14.0
852,FCuH1wdksA2DQ,Detroit,Lions,DET,17,25.00,13.26,8.68,6.14,9.65,...,2,13,0,9,70,1408,away,7.0,False,21.0
2232,SlVgURDFI7EXX,Pittsburgh,Steelers,PIT,25,-12.00,-1.35,5.02,-7.10,-3.14,...,1,7,3,9,111,1869,away,7.0,False,7.0
601,IR4gIYmh269EI,Cleveland,Browns,CLE,20,-14.00,-6.36,5.01,-9.70,-12.33,...,1,2,4,5,25,2063,away,6.0,False,14.0
2406,4M8AK5p0WXs1N,Seattle,Seahawks,SEA,32,2.35,0.17,8.95,-2.45,-1.63,...,3,29,3,9,66,2332,away,14.0,False,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972,CI5mhfNa0Zqd2,New Orleans,Saints,NO,19,-2.00,1.53,5.70,-1.50,-6.99,...,8,37,0,9,84,2450,away,14.0,False,14.0
765,KdjU81i7YSFE7,Dallas,Cowboys,DAL,19,-3.00,2.48,2.27,1.96,-5.93,...,6,43,1,5,55,1377,away,7.0,False,21.0
2574,QMAZH1v922YLu,Tampa Bay,Buccaneers,TB,20,-3.00,7.96,14.57,-5.52,-11.83,...,1,0,0,5,39,2126,away,7.0,False,7.0
935,FCuH1wdksA2DQ,Detroit,Lions,DET,31,-14.00,13.75,-2.14,16.75,-27.77,...,0,0,0,4,25,2039,away,6.0,False,6.0


In [3]:
team_info = {
    "ARI": {"city": "Glendale, AZ", "lat": 33.5276, "lon": -112.2626, "tz": "America/Phoenix"},
    "ATL": {"city": "Atlanta, GA", "lat": 33.7550, "lon": -84.3900, "tz": "America/New_York"},
    "BAL": {"city": "Baltimore, MD", "lat": 39.2780, "lon": -76.6227, "tz": "America/New_York"},
    "BUF": {"city": "Orchard Park, NY", "lat": 42.7738, "lon": -78.7862, "tz": "America/New_York"},
    "CAR": {"city": "Charlotte, NC", "lat": 35.2251, "lon": -80.8526, "tz": "America/New_York"},
    "CHI": {"city": "Chicago, IL", "lat": 41.8623, "lon": -87.6167, "tz": "America/Chicago"},
    "CIN": {"city": "Cincinnati, OH", "lat": 39.0954, "lon": -84.5161, "tz": "America/New_York"},
    "CLE": {"city": "Cleveland, OH", "lat": 41.5061, "lon": -81.6995, "tz": "America/New_York"},
    "DAL": {"city": "Arlington, TX", "lat": 32.7473, "lon": -97.0945, "tz": "America/Chicago"},
    "DEN": {"city": "Denver, CO", "lat": 39.7439, "lon": -105.0201, "tz": "America/Denver"},
    "DET": {"city": "Detroit, MI", "lat": 42.3400, "lon": -83.0456, "tz": "America/New_York"},
    "GB": {"city": "Green Bay, WI", "lat": 44.5013, "lon": -88.0622, "tz": "America/Chicago"},
    "HOU": {"city": "Houston, TX", "lat": 29.6847, "lon": -95.4107, "tz": "America/Chicago"},
    "IND": {"city": "Indianapolis, IN", "lat": 39.7601, "lon": -86.1639, "tz": "America/New_York"},
    "JAC": {"city": "Jacksonville, FL", "lat": 30.3239, "lon": -81.6373, "tz": "America/New_York"},
    "KC": {"city": "Kansas City, MO", "lat": 39.0490, "lon": -94.4849, "tz": "America/Chicago"},
    "LV": {"city": "Las Vegas, NV", "lat": 36.0909, "lon": -115.1830, "tz": "America/Los_Angeles"},
    "LAC": {"city": "Inglewood, CA", "lat": 33.9535, "lon": -118.3392, "tz": "America/Los_Angeles"},
    "LA": {"city": "Inglewood, CA", "lat": 33.9535, "lon": -118.3392, "tz": "America/Los_Angeles"},
    "MIA": {"city": "Miami Gardens, FL", "lat": 25.9580, "lon": -80.2389, "tz": "America/New_York"},
    "MIN": {"city": "Minneapolis, MN", "lat": 44.9740, "lon": -93.2581, "tz": "America/Chicago"},
    "NE": {"city": "Foxborough, MA", "lat": 42.0909, "lon": -71.2643, "tz": "America/New_York"},
    "NO": {"city": "New Orleans, LA", "lat": 29.9509, "lon": -90.0816, "tz": "America/Chicago"},
    "NYG": {"city": "East Rutherford, NJ", "lat": 40.8136, "lon": -74.0745, "tz": "America/New_York"},
    "NYJ": {"city": "East Rutherford, NJ", "lat": 40.8136, "lon": -74.0745, "tz": "America/New_York"},
    "PHI": {"city": "Philadelphia, PA", "lat": 39.9008, "lon": -75.1675, "tz": "America/New_York"},
    "PIT": {"city": "Pittsburgh, PA", "lat": 40.4468, "lon": -80.0158, "tz": "America/New_York"},
    "SEA": {"city": "Seattle, WA", "lat": 47.5952, "lon": -122.3316, "tz": "America/Los_Angeles"},
    "SF": {"city": "Santa Clara, CA", "lat": 37.4030, "lon": -121.9703, "tz": "America/Los_Angeles"},
    "TB": {"city": "Tampa, FL", "lat": 27.9759, "lon": -82.5033, "tz": "America/New_York"},
    "TEN": {"city": "Nashville, TN", "lat": 36.1665, "lon": -86.7713, "tz": "America/Chicago"},
    "WAS": {"city": "Landover, MD", "lat": 38.9078, "lon": -76.8644, "tz": "America/New_York"},
}

In [256]:
def calculate_distance(row):
    away = row['away_alias']
    home = row['home_alias']

    if away not in team_info or home not in team_info:
        return None

    away_coords = (team_info[away]['lat'], team_info[away]['lon'])
    home_coords = (team_info[home]['lat'], team_info[home]['lon'])

    return geodesic(away_coords, home_coords).miles

def compute_tz_change(row):
    try:
        away_tz = ZoneInfo(team_info[row['away_alias']]['tz'])
        home_tz = ZoneInfo(team_info[row['home_alias']]['tz'])
        dt_utc = datetime.combine(row['event_date'], datetime.min.time()).replace(tzinfo=ZoneInfo("UTC"))

        away_offset = dt_utc.astimezone(away_tz).utcoffset().total_seconds() / 3600
        home_offset = dt_utc.astimezone(home_tz).utcoffset().total_seconds() / 3600

        return abs(home_offset - away_offset)
    except Exception as e:
        # Optionally: print(f"Skipping row due to: {e}")
        return None

In [258]:
merged_df['tz_change'] = merged_df.apply(calculate_timezone_change, axis=1)
merged_df['away_travel_distance'] = merged_df.apply(calculate_distance, axis=1)

In [268]:
away_df['tz_change'] = away_df.apply(calculate_timezone_change, axis=1)
away_df['travel_distance'] = away_df.apply(calculate_distance, axis=1)

KeyError: 'home_alias'